Name: Tushar Muley

Assignment: Assignment 11

Date: February 13, 2022

In [8]:
# import libraries
import keras
from keras import layers
import numpy as np
import random
import sys


In [2]:
path = keras.utils.get_file(
'nietzsche.txt',
origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 1us/step
Corpus length: 600893


In [3]:
 # extract sequences of 60 characters
maxlen = 60

# sample a new sequence every three characters
step = 3

# hold the extracted sequences
sentences = []

# holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Sequences:', len(sentences))

# list of characters in eron data
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

# maps characters to their index
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')


# one-hot encode the characters into binary arrays
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



Sequences: 200278
Unique characters: 57
Vectorization...


#### Single-layer LSTM model for next-chracter prediction

In [5]:
# single-layer LSTM

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))


In [6]:
# model compilation configuration

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [7]:
# function to sample the next character

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [9]:
# train the model for 20 examples

for epoch in range(1, 20):
    print('epoch', epoch)

# fit the model for one iteration on the data
    model.fit(x, y, batch_size=128, epochs=1)

    # select a text seed at random

    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    # different sampling temperatures

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # generates 400 characters, from seed text
        for i in range(400):

            # one-hot encodes the characters genrated so far
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

        # samples the next character
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        
        generated_text += next_char
        generated_text = generated_text[1:]
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()



epoch 1
1565/1565 [==============================] - 199s 127ms/step - loss: 1.9554
--- Generating with seed: "sight of it would necessarily seduce and constrain to sympat"
------ temperature: 0.2
sight of it would necessarily seduce and constrain to sympath------ temperature: 0.5
ight of it would necessarily seduce and constrain to sympathy------ temperature: 1.0
ght of it would necessarily seduce and constrain to sympathy,------ temperature: 1.2
ht of it would necessarily seduce and constrain to sympathy, 
epoch 2
1565/1565 [==============================] - 198s 126ms/step - loss: 1.6127
--- Generating with seed: "y and dangerous isolation, as a saint: for it is just
holine"
------ temperature: 0.2
y and dangerous isolation, as a saint: for it is just
holines------ temperature: 0.5
 and dangerous isolation, as a saint: for it is just
holines,------ temperature: 1.0
and dangerous isolation, as a saint: for it is just
holines, ------ temperature: 1.2
nd dangerous isolation, as a saint

once more in the same orbit, however independent of earth-
epoch 17
1565/1565 [==============================] - 197s 126ms/step - loss: 1.3305
--- Generating with seed: "hicle for
your inclinations,--ye, too, really find the noise"
------ temperature: 0.2
hicle for
your inclinations,--ye, too, really find the noise ------ temperature: 0.5
icle for
your inclinations,--ye, too, really find the noise i------ temperature: 1.0
cle for
your inclinations,--ye, too, really find the noise it------ temperature: 1.2
le for
your inclinations,--ye, too, really find the noise its
epoch 18
1565/1565 [==============================] - 197s 126ms/step - loss: 1.3265
--- Generating with seed: "nifests is a consolation to the weak and suffering only in a"
------ temperature: 0.2
nifests is a consolation to the weak and suffering only in a ------ temperature: 0.5
ifests is a consolation to the weak and suffering only in a c------ temperature: 1.0
fests is a consolation to the weak and suffering only in a